In [7]:
import os
import psycopg2
from dotenv import load_dotenv
import mysql.connector
import boto3
import json
from datetime import datetime
import cv2
import matplotlib.pyplot as plt
import numpy as np



In [8]:

def import_from_S3(S3directory, filename, outputDirectory):

    # load environment variables from .env file
    load_dotenv()

    # access specific s3 bucket
    bucket_name = os.getenv('AWS_BUCKET')

    # create an S3 client
    s3 = boto3.client('s3')

    # list all of the buckets in your account
    response = s3.list_buckets()
    print(response)


    print(S3directory + filename)

    s3.download_file(bucket_name, S3directory + filename, outputDirectory + filename)


In [9]:

load_dotenv()
host=os.getenv('DB_HOST')

conn = mysql.connector.connect(
    host=os.getenv('DB_HOST'),
    user=os.getenv('DB_USERNAME'),
    password= os.getenv('DB_PASSWORD'),
    database=os.getenv('DB_DATABASE'),
    port=os.getenv('DB_PORT')
)

cursor = conn.cursor()

query = "select url from logograb_videos where id = 5"

cursor.execute(query)

result = cursor.fetchall()
video_url = result[0][0]
filename = video_url.replace('http://spect8-static.s3.amazonaws.com/', '')
print(filename)

#import_from_S3('', filename, '.\\visua\\videos\\')



7801171_2021_05_09_MOL_KBK_1st_Half_445d25.mp4


In [10]:
def getFrame(seconds, fps, video_url):
    cap = cv2.VideoCapture(video_url)
    cap.set(cv2.CAP_PROP_POS_FRAMES, seconds * fps)
    ret, frame = cap.read()
    return frame

In [11]:
def plot(frame, rect = None):
    #x, y, w, h = 150, 50, 200, 200
    if rect is not None:
        rect = np.array(rect, dtype=np.int32)
        color = (0, 0, 255)
        thickness = 2
        #frame = cv2.rectangle(frame, (x, y), (x+w, y+h), color, thickness)
        cv2.polylines(frame, [rect], True, (0, 0, 255), 2)
    
    # Convert the frame to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Plot the frame using matplotlib
    plt.imshow(frame)
    plt.axis("off")
    plt.show()

In [12]:
def video_to_frames(video, framecount = 100):

    # Get the total number of frames in the video
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Set the starting frame number
    frame_number = 0

    # Loop through each frame in the video
    while video.isOpened():
        # Read the next frame
        ret, frame = video.read()

        # Check if frame was read successfully
        if ret:
            # Save the frame as a JPEG image
            filename = f"frame_{frame_number:06d}.jpg"
            cv2.imwrite('./images/' + filename, frame)

            # Increment the frame number
            frame_number += 1
        else:
            break



In [13]:
def saveFrames(url, numberOfImages, startID):
    id = startID

    video = cv2.VideoCapture(url)
    number_of_fames = video.get(cv2.CAP_PROP_FRAME_COUNT)
    # Get the total number of frames in the video
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)

    # Get the frames per second (fps) of the video
    fps = video.get(cv2.CAP_PROP_FPS)

    # Calculate the length of the video in seconds
    video_length = frame_count / fps

    skip = video_length/numberOfImages
    for i in range(numberOfImages):
        #def getFrame(seconds, fps, video_url):

        frame = getFrame(seconds=i*skip, fps=25, video_url=url)
        print(i*skip)
        # plot(frame)
        cv2.imwrite('./images/' + str(id) + '.jpg', frame)

        id += 1

In [14]:
def two_tofour_coordinates(x1, y1, x2, y2):
    points = np.array([[x1, y2], [x1, y1], [x2, y1], [x2, y2]], np.int32)
    return points

In [19]:
saveFrames('C:\\Users\\thoma\\Documents\\20221106_Norge_Albania_Studio_LandslagKvinner.mp4', 100, 452)

0.0
21.717980143229166
43.43596028645833
65.1539404296875
86.87192057291666
108.58990071614583
130.307880859375
152.02586100260416
173.74384114583333
195.4618212890625
217.17980143229167
238.89778157552084
260.61576171875
282.33374186197915
304.0517220052083
325.7697021484375
347.48768229166666
369.2056624348958
390.923642578125
412.64162272135417
434.35960286458334
456.0775830078125
477.7955631510417
499.51354329427085
521.2315234375
542.9495035807291
564.6674837239583
586.3854638671875
608.1034440104166
629.8214241536458
651.539404296875
673.2573844401041
694.9753645833333
716.6933447265625
738.4113248697917
760.1293050130208
781.84728515625
803.5652652994792
825.2832454427083
847.0012255859375
868.7192057291667
890.4371858723958
912.155166015625
933.8731461588542
955.5911263020834
